# SCAPY
18/09/2024

## Cours
Ethernet -> MAC  
IP -> adresses IP  
Transport -> port  

## Code

#### Import

In [11]:
from scapy.all import *

#### Connaitre les interfaces disponibles

In [ ]:
# Affiche toutes les interfaces réseau disponibles
print(conf.ifaces)

In [ ]:
from scapy.all import *
sniff(count=10)

#### Voir les trames

TTL : 64 passages de routeur, (traceroute travaille dessus)

In [ ]:

def maFonction(trame):
    """
    Voir les trames
    """
    print(trame)
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, count=15)


Filtrer :

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="tcp", count=15)

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="ip and tcp 10.10.1.2", count=15)

Filtrage BPF : Berkeley Packet Filter  
* Filtrage rapide mais peu flexible

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="ip and tcp and src net 192.168.1.0/24", count=15)

Filtrage par fonction

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

def monFiltre(pkt):
    if(pkt.haslayer(TCP)): # Si trame à une couche TCP
        return True
    else:
        return False

sniff(iface="Software Loopback Interface 1", prn=maFonction, lfilter=monFiltre, count=15)

Affichage  
Adresse ip source  
Adresse Mac source

In [ ]:

def maFonction(trame):
    print("L'adresse ip source est :", trame[IP].src) # Détaille des trames
    #print("Trame TCP avec port 443 : ", trame[TCP].dport==443) # Détaille des trames
    #print("L'adresse Mac source est :", trame[Ether].src) # Détaille des trames
    #print("L'applicatif : ", trame[RAW].load)

def monFiltre(pkt):
    if(pkt.haslayer(TCP)): # Si trame à une couche TCP
        return True
    else:
        return False

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="tcp" ,lfilter=monFiltre, count=15)

#### Fabrication de trame

In [ ]:
a=Ether()
a
a.show()

Changement de mac

In [ ]:
a=Ether(src="00:01:03:04:05")
a.show()

Changement d'adresse Ip

In [ ]:
b=IP(dst="192.168.12.1",ttl=2)
b.show()

Réunir les couches

In [ ]:
c=a/b
c.show()

Couche TCP

In [ ]:
c=a/b/TCP(dport=80)
c.show()

In [ ]:
c=a/b/TCP(dport=80)/"GET index.php HTML/1.1\r\n"

Envoi de tram  
MAC : 
* sendp6() -> Envoi sans retour (ddos)
* sr1p6() -> Envoi un retour
* srp() -> Envoi reçoit retour

Couche Ip :
* send()
* sr1()
* sr()

In [14]:
send(Ether()/IP(dst="192.168.56.1")/ICMP())


Sent 1 packets.


In [15]:
retour = sr1(IP(dst="oteria.fr")/ICMP()/"test oteria")

Begin emission:
Finished sending 1 packets.

Received 4 packets, got 1 answers, remaining 0 packets


In [16]:
retour.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 39
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 249
  proto     = icmp
  chksum    = 0x24a7
  src       = 75.2.70.75
  dst       = 10.100.1.125
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0xafe7
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Raw ]### 
        load      = 'test oteria'
###[ Padding ]### 
           load      = '\x00\x00\x00'



Envoi destination à 5 ports/paquets

In [18]:
reponse,nonreponse =sr(IP(dst="oteria.fr")/TCP(dport=[21,22,23,80,443], flags="S"))

Begin emission:
Finished sending 5 packets.

Received 4311 packets, got 2 answers, remaining 3 packets


Plage de port

In [19]:
reponse,nonreponse =sr(IP(dst="oteria.fr")/TCP(dport=(21,80), flags="S"))

Begin emission:
Finished sending 60 packets.

Received 4624 packets, got 1 answers, remaining 59 packets


Plage d'adresse

In [20]:
reponse,nonreponse =sr(IP(dst=["oteria.fr","uphf.fr"])/TCP(dport=(21,80), flags="S"))

Begin emission:
Finished sending 120 packets.

Received 3945 packets, got 2 answers, remaining 118 packets


In [21]:
reponse[0], reponse[1]

(QueryAnswer(query=<IP  frag=0 proto=tcp dst=75.2.70.75 |<TCP  dport=http flags=S |>>, answer=<IP  version=4 ihl=5 tos=0x0 len=44 id=1 flags= frag=0 ttl=249 proto=tcp chksum=0x249d src=75.2.70.75 dst=10.100.1.125 |<TCP  sport=http dport=ftp_data seq=0 ack=0 dataofs=6 reserved=0 flags=SA window=65535 chksum=0x39 urgptr=0 options=[('MSS', 0)] |>>),
 QueryAnswer(query=<IP  frag=0 proto=tcp dst=194.167.248.41 |<TCP  dport=http flags=S |>>, answer=<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=52 proto=tcp chksum=0x801a src=194.167.248.41 dst=10.100.1.125 |<TCP  sport=http dport=ftp_data seq=493519669 ack=1 dataofs=6 reserved=0 flags=SA window=64240 chksum=0x356f urgptr=0 options=[('MSS', 1460)] |>>))

In [22]:
reponse[0][1], reponse[1][1]

(<IP  version=4 ihl=5 tos=0x0 len=44 id=1 flags= frag=0 ttl=249 proto=tcp chksum=0x249d src=75.2.70.75 dst=10.100.1.125 |<TCP  sport=http dport=ftp_data seq=0 ack=0 dataofs=6 reserved=0 flags=SA window=65535 chksum=0x39 urgptr=0 options=[('MSS', 0)] |>>,
 <IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=52 proto=tcp chksum=0x801a src=194.167.248.41 dst=10.100.1.125 |<TCP  sport=http dport=ftp_data seq=493519669 ack=1 dataofs=6 reserved=0 flags=SA window=64240 chksum=0x356f urgptr=0 options=[('MSS', 1460)] |>>)

In [23]:
reponse,nonreponse =sr(IP(dst="google.fr", ttl=(1,15))/TCP(flags="S"))

for var, var1 in reponse:
    print("ttl : ", var[IP].ttl, " Adresse IP :", var1[IP].src)

Begin emission:
Finished sending 15 packets.

Received 9854 packets, got 13 answers, remaining 2 packets
ttl :  1  Adresse IP : 10.100.1.254
ttl :  2  Adresse IP : 172.29.10.17
ttl :  3  Adresse IP : 185.91.90.252
ttl :  4  Adresse IP : 185.57.223.162
ttl :  5  Adresse IP : 5.226.1.176
ttl :  6  Adresse IP : 85.31.194.80
ttl :  7  Adresse IP : 195.234.35.34
ttl :  8  Adresse IP : 72.14.233.77
ttl :  9  Adresse IP : 216.239.48.1
ttl :  10  Adresse IP : 216.58.214.163
ttl :  11  Adresse IP : 216.58.214.163
ttl :  12  Adresse IP : 216.58.214.163
ttl :  13  Adresse IP : 216.58.214.163


Permet de faire du scan de port, détecter du bruit de force